In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import keras_cv
import tensorflow as tf

Using TensorFlow backend


/home/ubuntu/miniconda3/envs/oppai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = keras_cv.models.StableDiffusion(
    img_width=512, img_height=512, jit_compile=False
)

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


In [3]:
unet = model.diffusion_model

unet.summary()

Model: "diffusion_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 timestep_embedding (InputL  [(None, 320)]                0         []                            
 ayer)                                                                                            
                                                                                                  
 dense (Dense)               (None, 1280)                 410880    ['timestep_embedding[0][0]']  
                                                                                                  
 latent (InputLayer)         [(None, 64, 64, 4)]          0         []                            
                                                                                                  
 activation (Activation)     (None, 1280)                 0         ['dense[0][0]'] 

In [4]:
layers = ["spatial_transformer", "spatial_transformer_2", "spatial_transformer_4", "spatial_transformer_6"]

In [5]:
import tf2onnx

i = 0
# detect all layers with name starting with "spatial_transformer"
for layer in unet.layers:
    if layer.name in layers:
        inputs = [
            tf.keras.Input(shape=layer.input[0].shape.as_list()[1:], name="input_0"),
            tf.keras.Input(shape=layer.input[1].shape.as_list()[1:], name="context"),
        ]
        specs = [
            tf.TensorSpec([1]+inputs[0].shape.as_list()[1:], inputs[0].dtype, name=inputs[0].name),
            tf.TensorSpec([1]+inputs[1].shape.as_list()[1:], inputs[1].dtype, name=inputs[1].name),
        ]
        nchw = [inputs[0].name]

        outputs = layer(inputs)
        model_i = tf.keras.Model(inputs=inputs, outputs=outputs, name=layer.name)
        tf2onnx.convert.from_keras(
            model_i,
            input_signature=specs,
            inputs_as_nchw=nchw,
            opset=18,
            output_path=f'{layer.name}/model.onnx'
        )

In [7]:
inputs = [
    tf.keras.Input(shape=unet.input[0].shape.as_list()[1:], name="latent"),
    tf.keras.Input(shape=unet.input[1].shape.as_list()[1:], name="timestep_embedding"),
    tf.keras.Input(shape=unet.input[2].shape.as_list()[1:], name="context"),
]
specs = [
    tf.TensorSpec([1]+inputs[0].shape.as_list()[1:], inputs[0].dtype, name=inputs[0].name),
    tf.TensorSpec([1]+inputs[1].shape.as_list()[1:], inputs[1].dtype, name=inputs[1].name),
    tf.TensorSpec([1]+inputs[2].shape.as_list()[1:], inputs[2].dtype, name=inputs[2].name),
]
nchw = [inputs[0].name]

tf2onnx.convert.from_keras(
    unet,
    input_signature=specs,
    inputs_as_nchw=nchw,
    opset=18,
    output_path=f'unet.onnx',
    large_model=True
)

(ir_version: 8
 producer_name: "tf2onnx"
 producer_version: "1.16.1 15c810"
 graph {
   node {
     input: "timestep_embedding"
     input: "diffusion_model/dense/MatMul/ReadVariableOp__550"
     input: "diffusion_model/dense/BiasAdd/ReadVariableOp__551"
     output: "diffusion_model/dense/MatMul_Gemm__1601:0"
     name: "diffusion_model/dense/MatMul_Gemm__1601"
     op_type: "Gemm"
     attribute {
       name: "transA"
       i: 0
       type: INT
     }
     attribute {
       name: "transB"
       i: 0
       type: INT
     }
     domain: ""
   }
   node {
     input: "diffusion_model/res_block_20/activation_63/beta:0"
     input: "diffusion_model/dense/MatMul_Gemm__1601:0"
     output: "diffusion_model/activation/mul:0"
     name: "diffusion_model/activation/mul"
     op_type: "Mul"
   }
   node {
     input: "diffusion_model/activation/mul:0"
     output: "diffusion_model/activation/Sigmoid:0"
     name: "diffusion_model/activation/Sigmoid"
     op_type: "Sigmoid"
   }
   node {
